In [1]:
import pandas as pd
import numpy as np
import chess

import itertools

### Loading data and parsing the text file for moves, results and openings

In [2]:
game_list = [line for line in open('GMallboth.pgn')] 

all_moves_list = []
i = 11
while i <= (len(game_list)/100):           ###### NOTE: '/100' only using 1% of the data availible from grandmasters
    all_moves_list.append(game_list[i])
    i += 16
    
    
all_results = []
i = 13
while i <= (len(game_list)/100):
    all_results.append(game_list[i])
    i += 16
    
all_openings = []
i = 7
while i <= (len(game_list)/100):
    all_openings.append(game_list[i])
    i += 16

In [3]:
def get_move_list(move_string): #### input is a string 
    '''
    Takes a move string that was striped from a PGN format, and removes unwanted characters and conserves SAN format.
    
    Input: String of plain text moves in string format
    Output: List of SAN moves
    
    '''
    testing = move_string 
    testing = testing[0:-2]  ### remove last 2 characters = '\n'
    testing = testing.split('. ')
    
    for i in range(len(testing)):
        testing[i] = testing[i].split(' ')
    for i in testing:
        try:
            del(i[2])
        except:
            continue
    del(testing[0])

    simplelist = list(itertools.chain.from_iterable(testing))
    return(simplelist)

In [4]:
%%time
all_san_list = [get_move_list(game) for game in all_moves_list]

CPU times: user 211 ms, sys: 23 ms, total: 234 ms
Wall time: 237 ms


In [5]:
def white_results(result_string_list):
    '''
    Reformats and standardizes stripped result text from chess game in PGN/TXT format
    
    Input: list of strings - (results text with \n)
    Output: win, lose, draw in list format
    
    '''
    white_result_list = []
    for i in range(len(result_string_list)):
            result = result_string_list[i][:-1]
            result = result.replace('1-0', 'win')
            result = result.replace('0-1', 'lose')
            result = result.replace('1/2-1/2', 'draw')
            white_result_list.append(result)
    return(white_result_list)

In [6]:
%%time
white_test = white_results(all_results)

CPU times: user 4.43 ms, sys: 1.78 ms, total: 6.21 ms
Wall time: 11.9 ms


In [90]:
white_win = []
white_lose = []
white_draw = []

for i in range(len(white_test)):
    if white_test[i] == 'win':
        white_win.append(all_san_list[i])
    elif white_test[i] == 'lose':
        white_lose.append(all_san_list[i])
print(len(white_win), len(white_lose), len(white_draw))

871 521 0


In [8]:
from stockfish import Stockfish

stockfish = Stockfish('Desktop/stockfish-11-mac/Mac/stockfish-11-64')

In [9]:
import chess
import chess.engine

engine = chess.engine.SimpleEngine.popen_uci("Desktop/stockfish-11-mac 2/Mac/stockfish-11-64")

#board = chess.Board()
#info = engine.analyse(board, chess.engine.Limit(time=0.1))
#print("Score:", info["score"])
# Score: +95

board = chess.Board()
print(board)
#info = engine.analyse(board, chess.engine.Limit(depth=10))
#print("Score:", info["score"])
# Score: #+1
#print(info['depth'], info['nodes'], info['time'])
engine.quit()

r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R


In [10]:
test_games = all_san_list[0:10]
test_games[0][0]

'd4'

In [106]:
%%time
list_scores = []
engine = chess.engine.SimpleEngine.popen_uci("Desktop/stockfish-11-mac 2/Mac/stockfish-11-64")
for game in test_games:
    list_scores.append('___________________________')
    board = chess.Board()
    for i in game:
        info = engine.analyse(board, chess.engine.Limit(depth=10))
        list_scores.append(board)
        board.push_san(i)
        list_scores.append(info['score'])
### . chess.engine.PlayResult()

CPU times: user 4.53 s, sys: 185 ms, total: 4.71 s
Wall time: 13.4 s


In [107]:
list_scores[:25]

['___________________________',
 Board('5q1k/6P1/4Q3/5P1K/7P/8/8/8 b - - 0 63'),
 PovScore(Cp(+105), WHITE),
 Board('5q1k/6P1/4Q3/5P1K/7P/8/8/8 b - - 0 63'),
 PovScore(Cp(+16), BLACK),
 Board('5q1k/6P1/4Q3/5P1K/7P/8/8/8 b - - 0 63'),
 PovScore(Cp(+101), WHITE),
 Board('5q1k/6P1/4Q3/5P1K/7P/8/8/8 b - - 0 63'),
 PovScore(Cp(+60), BLACK),
 Board('5q1k/6P1/4Q3/5P1K/7P/8/8/8 b - - 0 63'),
 PovScore(Cp(+29), WHITE),
 Board('5q1k/6P1/4Q3/5P1K/7P/8/8/8 b - - 0 63'),
 PovScore(Cp(-21), BLACK),
 Board('5q1k/6P1/4Q3/5P1K/7P/8/8/8 b - - 0 63'),
 PovScore(Cp(+51), WHITE),
 Board('5q1k/6P1/4Q3/5P1K/7P/8/8/8 b - - 0 63'),
 PovScore(Cp(-13), BLACK),
 Board('5q1k/6P1/4Q3/5P1K/7P/8/8/8 b - - 0 63'),
 PovScore(Cp(+107), WHITE),
 Board('5q1k/6P1/4Q3/5P1K/7P/8/8/8 b - - 0 63'),
 PovScore(Cp(-25), BLACK),
 Board('5q1k/6P1/4Q3/5P1K/7P/8/8/8 b - - 0 63'),
 PovScore(Cp(+96), WHITE),
 Board('5q1k/6P1/4Q3/5P1K/7P/8/8/8 b - - 0 63'),
 PovScore(Cp(-52), BLACK)]

In [108]:
duck =[]
for i in range(len(list_scores)):
    try:
        if i % 2:
            duck.append((int(str(list_scores[i])), 'white'))
        else:
            duck.append((int(str(list_scores[i])), 'black'))
    except:
        continue

In [105]:
duck

[(105, 'black'),
 (16, 'black'),
 (101, 'black'),
 (60, 'black'),
 (29, 'black'),
 (-21, 'black'),
 (51, 'black'),
 (-13, 'black'),
 (107, 'black'),
 (-25, 'black'),
 (96, 'black'),
 (-52, 'black'),
 (71, 'black'),
 (-33, 'black'),
 (126, 'black'),
 (-39, 'black'),
 (137, 'black'),
 (-49, 'black'),
 (52, 'black'),
 (-15, 'black'),
 (67, 'black'),
 (-21, 'black'),
 (100, 'black'),
 (3, 'black'),
 (61, 'black'),
 (-13, 'black'),
 (93, 'black'),
 (-37, 'black'),
 (161, 'black'),
 (-56, 'black'),
 (123, 'black'),
 (-63, 'black'),
 (89, 'black'),
 (-79, 'black'),
 (121, 'black'),
 (-19, 'black'),
 (97, 'black'),
 (-20, 'black'),
 (68, 'black'),
 (-17, 'black'),
 (169, 'black'),
 (-94, 'black'),
 (87, 'black'),
 (-27, 'black'),
 (86, 'black'),
 (13, 'black'),
 (59, 'black'),
 (0, 'black'),
 (122, 'black'),
 (-16, 'black'),
 (33, 'black'),
 (36, 'black'),
 (75, 'black'),
 (35, 'black'),
 (-4, 'black'),
 (18, 'black'),
 (83, 'black'),
 (54, 'black'),
 (44, 'black'),
 (20, 'black'),
 (93, 'blac

In [ ]:
engine = chess.engine.SimpleEngine.popen_uci("Desktop/stockfish-11-mac 2/Mac/stockfish-11-64")

In [ ]:
engine

In [25]:
board = chess.Board()
board.push_san('d4')
print(board)


r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . P . . . .
. . . . . . . .
P P P . P P P P
R N B Q K B N R
